In [1]:
# Зависимости
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import ColumnTransformer

from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier, plot_tree

from sklearn.metrics import mean_squared_error, f1_score

In [2]:
# Генерируем уникальный seed
my_code = "Petrov"
seed_limit = 2 ** 32
my_seed = int.from_bytes(my_code.encode(), "little") % seed_limit

In [3]:
# Читаем данные из файла
example_data = pd.read_csv("datasets/Fish.csv")

In [4]:
example_data.head()

,Species,Weight,Length1,Length2,Length3,Height,Width
0,Bream,242.0,23.2,25.4,30.0,11.5200,4.0200
1,Bream,290.0,24.0,26.3,31.2,12.4800,4.3056
2,Bream,340.0,23.9,26.5,31.1,12.3778,4.6961
3,Bream,363.0,26.3,29.0,33.5,12.7300,4.4555
4,Bream,430.0,26.5,29.0,34.0,12.4440,5.1340


In [5]:
# Определим размер валидационной и тестовой выборок
val_test_size = round(0.2*len(example_data))
print(val_test_size)

32


In [6]:
# Создадим обучающую, валидационную и тестовую выборки
random_state = my_seed
train_val, test = train_test_split(example_data, test_size=val_test_size, random_state=random_state)
train, val = train_test_split(train_val, test_size=val_test_size, random_state=random_state)
print(len(train), len(val), len(test))

95 32 32


In [7]:
# Значения в числовых столбцах преобразуем к отрезку [0,1].
# Для настройки скалировщика используем только обучающую выборку.
num_columns = ['Weight', 'Length1', 'Length2', 'Length3', 'Height', 'Width']

ct = ColumnTransformer(transformers=[('numerical', MinMaxScaler(), num_columns)], remainder='passthrough')
ct.fit(train)

ColumnTransformer(remainder='passthrough',
                  transformers=[('numerical', MinMaxScaler(),
                                 ['Weight', 'Length1', 'Length2', 'Length3',
                                  'Height', 'Width'])])

In [8]:
# Преобразуем значения, тип данных приводим к DataFrame
sc_train = pd.DataFrame(ct.transform(train))
sc_test = pd.DataFrame(ct.transform(test))
sc_val = pd.DataFrame(ct.transform(val))

In [9]:
# Устанавливаем названия столбцов
column_names = num_columns + ['Species']
sc_train.columns = column_names
sc_test.columns = column_names
sc_val.columns = column_names

In [10]:
sc_train

,Weight,Length1,Length2,Length3,Height,Width,Species
0,0.0,0.0,0.0,0.0,0.0,0.001659,Smelt
1,0.077626,0.204082,0.217391,0.210993,0.229791,0.339825,Perch
2,0.506849,0.459184,0.463138,0.455674,0.560283,0.950483,Perch
3,0.098326,0.244898,0.255198,0.276596,0.327489,0.38418,Roach
4,0.512938,0.54898,0.557656,0.544326,0.5862,0.851922,Perch
...,...,...,...,...,...,...,...
90,0.299848,0.342857,0.362949,0.406028,0.718722,0.590554,Bream
91,0.589041,0.559184,0.57656,0.608156,0.981066,0.800575,Bream
92,0.068493,0.204082,0.217391,0.210993,0.243627,0.323024,Perch
93,0.216438,0.332653,0.349716,0.388298,0.633382,0.472875,Bream


In [11]:
# Задание №1 - анализ деревьев принятия решений в задаче регрессии
# https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html
# criterion : {“mse”, “friedman_mse”, “mae”, “poisson”}, default=”mse”
# splitter : {“best”, “random”}, default=”best”
# max_depth : int, default=None
# min_samples_split : int or float, default=2
# min_samples_leaf : int or float, default=1

In [12]:
# Выбираем 4 числовых переменных, три их них будут предикторами, одна - зависимой переменной
n = 4
labels = random.sample(num_columns, n)

y_label = labels[0]
x_labels = labels[1:]

print(x_labels)
print(y_label)

['Height', 'Weight', 'Length1']
Width


In [13]:
# Отберем необходимые параметры
x_train = sc_train[x_labels]
x_test = sc_test[x_labels]
x_val = sc_val[x_labels]

y_train = sc_train[y_label]
y_test = sc_test[y_label]
y_val = sc_val[y_label]

In [14]:
x_train

,Height,Weight,Length1
0,0.0,0.0,0.0
1,0.229791,0.077626,0.204082
2,0.560283,0.506849,0.459184
3,0.327489,0.098326,0.244898
4,0.5862,0.512938,0.54898
...,...,...,...
90,0.718722,0.299848,0.342857
91,0.981066,0.589041,0.559184
92,0.243627,0.068493,0.204082
93,0.633382,0.216438,0.332653


In [15]:
# Создайте 4 модели с различными критериями ветвления criterion: 'mse', 'friedman_mse', 'mae', 'poisson'.
# Решите получившуюся задачу регрессии с помощью созданных моделей и сравните их эффективность.
# При необходимости применяйте параметры splitter, max_depth, min_samples_split, min_samples_leaf
# Укажите, какая модель решает задачу лучше других.
criterion_list = ['mse', 'friedman_mse', 'mae', 'poisson']
r_model_list = []
for i in range(len(criterion_list)):
    r_model_list.append(DecisionTreeRegressor(criterion=criterion_list[i], random_state=random_state))
    r_model_list[i].fit(x_train, y_train)
    print(i, r_model_list[i].score(x_val, y_val))

0 0.8938304407502621
1 0.8980701395168629
2 0.88186369651017
3 0.8653476895964565


In [16]:
test_pred = r_model_list[1].predict(x_test)
print(mean_squared_error(y_test, test_pred), r_model_list[1], sep='\n')

0.005921226964848853
DecisionTreeRegressor(criterion='friedman_mse', random_state=1920230736)


In [17]:
# Задание №2 - анализ деревьев принятия решений в задаче классификации
# https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html
# criterion : {“gini”, “entropy”}, default=”gini”
# splitter : {“best”, “random”}, default=”best”
# max_depth : int, default=None
# min_samples_split : int or float, default=2
# min_samples_leaf : int or float, default=1

In [18]:
# Выбираем 2 числовых переменных, которые будут параметрами элементов набора данных
# Метка класса всегда 'Species'
n = 2
x_labels = random.sample(num_columns, n)
y_label = 'Species'

print(x_labels)
print(y_label)

['Length2', 'Weight']
Species


In [19]:
# Отберем необходимые параметры
x_train = sc_train[x_labels]
x_test = sc_test[x_labels]
x_val = sc_val[x_labels]

y_train = sc_train[y_label]
y_test = sc_test[y_label]
y_val = sc_val[y_label]

In [20]:
x_train

,Length2,Weight
0,0.0,0.0
1,0.217391,0.077626
2,0.463138,0.506849
3,0.255198,0.098326
4,0.557656,0.512938
...,...,...
90,0.362949,0.299848
91,0.57656,0.589041
92,0.217391,0.068493
93,0.349716,0.216438


In [21]:
# Создайте 4 модели с различными критериями ветвления criterion : 'gini', 'entropy' и splitter : 'best', 'random'.
# Решите получившуюся задачу классификации с помощью созданных моделей и сравните их эффективность.
# При необходимости применяйте параметры max_depth, min_samples_split, min_samples_leaf
# Укажите, какая модель решает задачу лучше других.
import itertools
criterion = list(itertools.product(['gini', 'entropy'], ['best', 'random']))
c_model_list = []
for i in range(0,len(criterion)):
    c_model_list.append(DecisionTreeClassifier(criterion[i][0], criterion[i][1], random_state=random_state))
    c_model_list[i].fit(x_train, y_train)
    print(i, c_model_list[i].score(x_val, y_val))

0 0.4375
1 0.59375
2 0.4375
3 0.59375


/srv/conda/envs/notebook/lib/python3.7/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass criterion=gini, splitter=best as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)
/srv/conda/envs/notebook/lib/python3.7/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass criterion=gini, splitter=random as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)
/srv/conda/envs/notebook/lib/python3.7/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass criterion=entropy, splitter=best as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)
/srv/conda/envs/notebook/lib/python3.7/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass criterio

In [22]:
test_pred = c_model_list[1].predict(x_test)
f1 = f1_score(y_test, test_pred, average='weighted')
print(f1,  c_model_list[1], sep='\n')

0.572539592760181
DecisionTreeClassifier(random_state=1920230736, splitter='random')
